<a href="https://colab.research.google.com/github/sandipanbasu/aiml-capstone/blob/master/Mrc_BERT_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 757kB 2.8MB/s 
     |████████████████████████████████| 3.0MB 15.7MB/s 
     |████████████████████████████████| 890kB 36.3MB/s 
     |████████████████████████████████| 1.1MB 27.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=a02619cf3a529da7e433d475c54e4e57f0f0254d4fdb0a8663382419431f3266
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import BertTokenizer, TFBertForQuestionAnswering

berthugtokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
berthugmodel = TFBertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

In [ ]:
berthugmodel.summary()

In [ ]:
def predBERTHuggingface(question, context):  
  input_dict = berthugtokenizer(question, context, return_tensors='tf')
  start_scores, end_scores = berthugmodel(input_dict)
  all_tokens = berthugtokenizer.convert_ids_to_tokens(input_dict["input_ids"].numpy()[0])
  # print(all_tokens)
  answer = ' '.join(all_tokens[tf.math.argmax(start_scores, 1)[0] : tf.math.argmax(end_scores, 1)[0]+1])
  # print(answer)
  # Find the tokens with the highest `start` and `end` scores.
  answer_start = np.argmax(start_scores)
  answer_end = np.argmax(end_scores)
  # Combine the tokens in the answer and print it out.
  answer = ' '.join(all_tokens[answer_start:answer_end+1])
  # Start with the first token.
  answer = all_tokens[answer_start]

  # Select the remaining answer tokens and join them with whitespace.
  for i in range(answer_start + 1, answer_end + 1):      
      # If it's a subword token, then recombine it with the previous token.
      if all_tokens[i][0:2] == '##':
          answer += all_tokens[i][2:]    
      # Otherwise, add a space then the token.
      else:
          answer += ' ' + all_tokens[i]

  return answer 

In [ ]:
import textwrap
wrapper = textwrap.TextWrapper(width=80) 
# print(wrapper.fill(cnndf['context'].iloc[0]))
q1, c1  = "what was the death toll", "With a record 15968 coronavirus cases reported in the past 24 hours,the total count in India crossed the 4.5 lakh mark. The death toll has gone up to 14476"
q2, c2 = 'how many cases in mumbai', 'Delhi, which already is the second worst hit state in terms of coronavirus caseload and fatalities, overtook Mumbai after the number of cases soared by 3788 to touch 70390. Mumbai has so far recorded 69625 cases, according to official figures.'

# input_dict = berthugtokenizer(text=[q1,q2],text_pair =[c1,c2],is_pretokenized=True, return_tensors='tf')
# print(input_dict)

# input_dict_noar = berthugtokenizer(q2,c2, return_tensors='tf')
# print(input_dict_noar)


# all_tokens = berthugtokenizer.convert_ids_to_tokens(input_dict["input_ids"][0])
# print(all_tokens)

# all_tokens = berthugtokenizer.convert_ids_to_tokens(input_dict_noar["input_ids"][0])
# print(all_tokens)

print(predBERTHuggingface(q1,c1))


In [ ]:
# bert_pred=pd.DataFrame(columns=['c','q','true','pred'])
test = load_test_data(name='test-withstopwordspunct.csv')
bert_h_pred = test[['id','context','question','answer','plausible_answer']].copy()
preds = []
strategy = tf.distribute.MirroredStrategy()
# a better way is to batch its and tokenize 
# for a batch a record at a time and do inference 
# I was able to do for deeppavlov, for hugginface I felt lazy
with strategy.scope():
  for i in tqdm(range(test.shape[0])):
    try:
      p = ''
      p = predBERTHuggingface(test['question'].iloc[i],test['context'].iloc[i])
    except:
      print('error in prediction')
    preds = preds + [p]

bert_h_pred['prediction'] = preds

bert_h_pred.head()
# remove na pls
bert_h_pred.loc[bert_h_pred['prediction'].isna(), 'prediction'] = ''
bert_h_pred.to_csv(model_path + 'bert/bert-huggingface-results.csv')